In [1]:
import pandas as pd


def add_tcrspep(df):
    df["tcrs_pep"] = df["tcra"] + ":" + df["tcrb"] + "&"  + df["peptide"] 
    return df

def renamecolumns(df):
    dic = {'TRA':"tcra", "TRB":"tcrb", 
           "TRAV":"va", "TRAJ":"ja",
              "TRBV":"vb", "TRBJ":"jb",
                "T-Cell-Type":"t_cell_type",
                "Peptide":"peptide", "MHC":"mhc"}
    inverse_dic = {v:k for k,v in dic.items()}
           
    return df.rename(columns=dic)


train_mcpas = pd.DataFrame(pd.read_pickle("./Samples/mcpas_train_samples.pickle")).query('tcra!="UNK"')
train_mcpas = add_tcrspep(train_mcpas)
train_vdjdb = pd.DataFrame(pd.read_pickle("./Samples/vdjdb_no10x_train_samples.pickle")).query('tcra!="UNK"')
train_vdjdb = add_tcrspep(train_vdjdb)


In [2]:
pd.read_parquet("./mcpas_results.parquet").head()

,TRA,TRB,TRAV,TRAJ,TRBV,TRBJ,T-Cell-Type,Peptide,protein,MHC,sign,Score
0,CAASNYGSSGNKLIF,CTCSAGTGGYNYAEQFF,TRAV1-01,TRAJ3-01,TRBV1-01,TRBJ2-1:01,CD8,SSPPMFRV,M38,H-2Kb,0,9.213172e-01
1,UNK,CASSQWPGHQPQHF,None,None,TRBV4-1,TRBJ1-5,CD8,LPRRSGAAGA,NP177,HLA-B7,0,7.332439e-01
2,CAEIPRSPMFSGGYNKLIF,CASSIFTRRGYEQYF,TRAV5,None,TRBV28,None,None,SSLENFRAYV,PA,H-2Db,0,1.404658e-08
3,UNK,CSGRSKGAVNEKLFF,None,None,TRBV29-1,TRBJ1-4,CD8,NLVPMVATV,pp65,HLA-A2,0,6.738620e-01
4,UNK,CASSTGDYAEQF,None,None,TRBV6,TRBJ2-1,CD8,LPRRSGAAGA,NP177,HLA-B7,0,1.168027e-06


In [3]:
# df_mcpas = pd.read_parquet("./mcpas_results.parquet")
# df_vdjdb = pd.read_parquet("./vdjdb_results.parquet")

df_mcpas = pd.read_parquet("./mcpas_onlyseqsresults.parquet")
df_vdjdb = pd.read_parquet("./vdjdb_onlyseqsresults.parquet")



df_mcpas = renamecolumns(df_mcpas).query('tcra!="UNK"')
df_mcpas = add_tcrspep(df_mcpas)

df_vdjdb = renamecolumns(df_vdjdb).query('tcra!="UNK"')
df_vdjdb = add_tcrspep(df_vdjdb)

In [4]:
from sklearn.metrics import roc_auc_score, average_precision_score

def eval_df(df):
    print("len(df)", len(df))
    print("pos rate", df.sign.mean())
    yt = df.sign
    yp = df.Score
    print("roc", roc_auc_score(yt,yp))    
    print("avp", average_precision_score(yt,yp))

In [5]:
eval_df(df_mcpas[~df_mcpas.tcrs_pep.isin(train_mcpas.tcrs_pep)])

len(df) 4729
pos rate 0.1511947557623176
roc 0.8128513489500035
avp 0.39786160973452506


In [6]:
# eval_df(df_mcpas)

eval_df(df_vdjdb[~df_vdjdb.tcrs_pep.isin(train_vdjdb.tcrs_pep)])


len(df) 4010
pos rate 0.15037406483790525
roc 0.844517019637163
avp 0.48111121905385096
